# Interior Point Method

Pada dokumen ini, akan dijabarkan beberapa poin penting perihal kode interior point method untuk mencari solusi program linear

\begin{align}
&\min c^T X \\
&\text{ subject to } \\
&AX = b, \, X\geq 0 
\end{align}

dengan
\begin{equation}
A = \begin{bmatrix}
    1 & 1 & 0\\
    0 & 1 & 1
\end{bmatrix},
\end{equation}

$b = [2, 2]^T$ dan $c = [-1, -2, 0]^T$.

Pertama-tama, akan dijabarkan kode untuk menentukan aproksimasi awal dari interior point method, $X^{(0)}$

In [4]:
import numpy as np

def initial_point(A, b, c):
    """
    Find the initial point for interior point method.
    
    Parameters:
        A (numpy.ndarray): Constraint matrix (m x n)
        b (numpy.ndarray): RHS vector (m x 1)
        c (numpy.ndarray): Objective coefficients (n x 1)

    Return:
        x (numpy.ndarray): Initial points
    """
    
    x = A.T @ np.linalg.inv(A @ A.T) @ b # (Dim = n)
    _lambda = np.linalg.inv(A @ A.T) @ A @ c  # Dual variables (Lagrange multipliers) (Dim = m)
    s = c - A.T @ _lambda  # Slack variables (Dim = n)

    delta_x = np.max([0, -1.5 * np.min(x)])
    delta_s = np.max([0, -1.5 * np.min(x)])
    
    x += delta_x
    s += delta_s

    delta_x2 = 0.5 * (x.T @ s)/np.sum(s)
    delta_s2 = 0.5 * (x.T @ s)/np.sum(x)

    x += delta_x2
    s += delta_s2

    return x, _lambda, s

Setelah itu, program utama dari interior point method dapat dilihat pada kode di bawah ini.

In [6]:
def interior_point_method(A, b, c, tol=1e-6, max_iter=100):
    """
    Solves the LP problem: minimize c^T x
    subject to Ax = b, x >= 0

    Parameters:
        A (numpy.ndarray): Constraint matrix (m x n)
        b (numpy.ndarray): RHS vector (m x 1)
        c (numpy.ndarray): Objective coefficients (n x 1)
        tol (float): Convergence tolerance
        max_iter (int): Maximum number of iterations

    Returns:
        x (numpy.ndarray): Optimal solution
        status (str): Solution status ('Optimal' or 'Failed')
    """
    
    m, n = A.shape
    # Initialize initial point
    x, _lambda, s = initial_point(A, b, c)
    
    for iteration in range(max_iter):

        k = iteration + 1
        
        # Build the KKT matrix
        X = np.diag(x)
        S = np.diag(s)

        KKT = np.block([
            [np.zeros((n, n)), A.T, np.eye((n))],
            [A, np.zeros((m, m)), np.zeros((m, n))],
            [S, np.zeros((n, m)), X]
        ])
        
        # Residuals
        r_b = A @ x - b
        r_c = A.T @ _lambda + s - c

        # Build the RHS vector
        rhs = -np.concatenate([-r_c, -r_b, - np.diag(X @ S)])

        # Compute the central path parameter
        mu = np.dot(x, s) / n

        # Solve the linear system for affine search directions
        daff = np.linalg.solve(KKT, rhs)
        dx_aff = daff[:n]
        dlambda_aff = daff[n:n+m]
        ds_aff = daff[-n:]

        # Construct alpha affine (primal and dual) and duality measure affine
        # The default value: 1
        alpha_aff_primal = 1
        alpha_aff_dual = 1
        
        idx_x = np.where(dx_aff < 0)[0]
        if idx_x.size > 0:
            alpha_aff_primal = min(1, np.min(-x[idx_x] / dx_aff[idx_x]))
        idx_s = np.where(ds_aff < 0)[0]
        if idx_s.size > 0:
            alpha_aff_dual = min(1, np.min(-s[idx_s] / ds_aff[idx_s]))
        
        mu_aff = (x + alpha_aff_primal * dx_aff).T @ (s + alpha_aff_dual * ds_aff) / n

        # Define centering parameter
        sigma = (mu_aff / mu) ** 3
        
        # Solve the second linear system
        X_aff = np.diag(dx_aff)
        S_aff = np.diag(ds_aff)
        rhs_aff = -np.concatenate([-r_c, -r_b, - np.diag(X @ S) - np.diag(X_aff @ S_aff) + sigma * mu])
        
        d = np.linalg.solve(KKT, rhs_aff)
        dx = d[:n]
        dlambda = d[n:n+m]
        ds = d[-n:]

        # Find the rate alpha
        # The default value: 1
        alpha_primal = 1
        alpha_dual = 1
        eta = 1 - 0.1 ** k
        
        idx_x = np.where(dx < 0)[0]
        if idx_x.size > 0:
            alpha_primal_max = np.min(-x[idx_x] / dx[idx_x])
            alpha_primal = min(1, eta * alpha_primal_max)
        idx_s = np.where(ds_aff < 0)[0]
        if idx_s.size > 0:
            alpha_dual_max = np.min(-s[idx_s] / ds[idx_s])
            alpha_dual = min(1, eta * alpha_dual_max)

        # Check for convergence
        if np.linalg.norm(np.concatenate([alpha_primal * dx, alpha_dual * dlambda, alpha_dual * ds])) < tol:
            return x, 'Optimal'
        
        # Update variables
        x += alpha_primal * dx
        _lambda += alpha_dual * dlambda
        s += alpha_dual * ds

    return x, 'Failed'  # If max_iter is reached

# Example usage
if __name__ == "__main__":
    # Problem data
    A = np.array([[1, 1, 0], [0, 1, 1]])
    b = np.array([2, 2])
    c = np.array([-1, -2, 0])

    # Solve the LP
    x_opt, status = interior_point_method(A, b, c)

    print("Status:", status)
    print("Optimal solution:", x_opt)


Status: Optimal
Optimal solution: [5.69357458e-16 2.00000000e+00 3.25933729e-18]


Bandingkan dengan solusi menggunakan package PuLP

In [37]:
from pulp import *
x = LpVariable("x", 0)
y = LpVariable("y", 0)
z = LpVariable("z", 0)
prob = LpProblem("myProblem", LpMinimize)

prob += x + y == 2
prob += y + z == 2
prob += -x -2*y

status = prob.solve()
print(LpStatus[status])
value(x), value(y), value(z)

Optimal


(0.0, 2.0, 0.0)